In [228]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

# Pablo Marcos y Dionisio Perez

# Importamos Librerias
import numpy as np
import collections
from abc import ABCMeta,abstractmethod
from scipy.stats import norm

#from sklearn import datasets
import sklearn.naive_bayes as nb
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing 

In [229]:

class Datos(object):
    """Clase para leer y almacenar los datos de los ficheros .data proporcionados

    Attributes:
        ndatos (int): Numero de entradas de nuestro conjunto de datos
        nAtributos (int): Numero de atributos de cada dato
        nombreAtributos (list): Lista con los nombres de los atributos
        tipoAtributos (list): Lista con string representando el tipo de cada atributo
        nominalAtributos (list): Lista con True en las posiciones de los atributos nominales
        diccionarios (list): Lista de diccionarios con el valor de cada uno de los atributos nominales
        datos (numpy.ndarray) : Matrix ndatosxnAtributos con los datos recolectados y los atributos
            nominales traducidos.
    """

    TiposDeAtributos=('Continuo','Nominal')

    def __init__(self, nombreFichero, cast=None):
        """Constructor de la clase Datos

        Args:
            nombreFichero (str): path del fichero de datos a cargar
            cast (np.dtype, opcional) : Si se especifica la matriz de datos se
                casteara al tipo especificado, en otro caso si todos los atributos
                son nominales se almacenaran en tipo entero y si hay algun dato
                continuo en tipo float.
        """

        # Abrimos el fichero y procesamos la cabecera
        with open(nombreFichero) as f:

            # Guardamos el numero de datos
            self.nDatos = int(f.readline())

            # Guardamos la lista de nombres de atributos
            self.nombreAtributos = f.readline().replace('\n','').split(",")

            # Guardamos la lista de atributos
            self.tipoAtributos = f.readline().replace('\n','').split(",")

            # Numero de atributos
            self.nAtributos = len(self.tipoAtributos)

            # Comprobacion atributos
            if any(atr not in Datos.TiposDeAtributos for atr in self.tipoAtributos):
                raise ValueError("Tipo de atributo erroneo")

            # Guardamos True en las posiciones de atributos nominales
            self.nominalAtributos = [atr == 'Nominal' for atr in self.tipoAtributos]

        # Leemos los datos de numpy en formate string para los datos nominales
        datosNominales = np.genfromtxt(nombreFichero, dtype='S', skip_header=3, delimiter=',')

        # Inicializamos los diccionarios con los distintos valores de los atributos
        self._inicializarDiccionarios(datosNominales)

        # Transformamos los datos nominales en datos numericos empleando los diccionarios
        for i, nominal in enumerate(self.nominalAtributos):
            if nominal:
                datosNominales[:,i] = np.vectorize(self.diccionarios[i].get)(datosNominales[:,i])

        # Convertimos la matriz a tipo numerico, en caso de no especificarse
        # Si todos los atributos son nominales usamos el tipo np.int para ahorrar espacio
        # Si hay datos continuos lo guardamos en tipo np.float
        if cast == None: cast = np.int if all(self.nominalAtributos) else np.float
        self.datos = datosNominales.astype(cast)

        # Convertimos los nombres nominales a string en vez de dejarlos en bytes
        diccionarios_aux = []
        for d in self.diccionarios:
            aux = {}
            for k in d: aux[k.decode('utf-8')] = d[k]
            diccionarios_aux.append(aux)

        self.diccionarios = diccionarios_aux

    def _inicializarDiccionarios(self, datos):
        """Funcion interna para inicializar los diccionarios buscando todos
            los valores que toman los atributos en la matriz de datos"""

        self.diccionarios = []

        for i, nominal in enumerate(self.nominalAtributos):

            if not nominal: # Incluimos diccionarios vacios en los datos no nominales
                self.diccionarios.append({})
            else:
                # Buscamos todos los valores distintos por atributo y creamos el diccionario
                values = np.unique(datos[:,i])
                values.sort()
                self.diccionarios.append({k: v for v, k in enumerate(values)})

    def extraeDatos(self, idx):
        return self.datos[idx]
    
    def __getitem__(self, idx):
        return self.extraeDatos(idx)
    
    def __len__(self):
        return self.nDatos
    
    def __yield__(self):
        for i in range(len(self)):
            yield self.datos[i]

    

In [230]:
from abc import ABCMeta,abstractmethod


class Particion:
  
    def __init__(self, train=[], test=[]):
        self.indicesTrain= train
        self.indicesTest= test
    
    def __str__(self):
        return "Particion:\nTrain: {}\nTest:  {}".format(str(self.indicesTrain),str(self.indicesTest)) 

#####################################################################################################

class EstrategiaParticionado:
  
      # Clase abstracta
    __metaclass__ = ABCMeta
  
    def __init__(self, nombre="null"):
        self.nombreEstrategia=nombre
        self.numeroParticiones=0
        self.particiones=[]
    
    def __call__(self, datos):
        return self.creaParticiones(datos)
    
    def __iter__(self):
        for part in self.particiones:
            yield part
  
    @abstractmethod
    # TODO: esta funcion deben ser implementadas en cada estrategia concreta  
    def creaParticiones(self,datos,seed=None):
        pass

#####################################################################################################

class ValidacionSimple(EstrategiaParticionado):
    """Crea particiones segun el metodo tradicional 
    de division de los datos segun el porcentaje deseado."""
    
    def __init__(self,porcentaje=.75):

        self.porcentaje = porcentaje
        super().__init__("Validacion Simple con {}\% de entrenamiento".format(100*porcentaje))


    def creaParticiones(self,datos,seed=None):    
        np.random.seed(seed)

        self.numeroParticiones = 1

        # Generamos una permutacion de los indices
        indices = np.arange(datos.nDatos)
        np.random.shuffle(indices)

        # Separamos en base al porcentaje necesario
        l = int(datos.nDatos*self.porcentaje)
        self.particiones = [Particion(indices[:l], indices[l:])]

        return self.particiones
    
      
#####################################################################################################      
class ValidacionCruzada(EstrategiaParticionado):
    
    def __init__(self, k=1):
        self.k = k
        super().__init__("Validacion Cruzada con {} particiones".format(k))
  
  # Crea particiones segun el metodo de validacion cruzada.
  # El conjunto de entrenamiento se crea con las nfolds-1 particiones
  # y el de test con la particion restante
  # Esta funcion devuelve una lista de particiones (clase Particion)
    def creaParticiones(self,datos,seed=None):   
        np.random.seed(seed)
        
        self.numeroParticiones = self.k
        # Tam de cada bloque
        l = int(datos.nDatos/self.k)
        
        # Generamos una permutacion de los indices
        indices = np.arange(datos.nDatos)
        np.random.shuffle(indices)
        self.particiones = []
        
        
        for i in range(self.k):

            train = np.delete(indices, range(i*l,(i+1)*l))
            test =  indices[i*l:(i+1)*l-1]
            self.particiones.append(Particion(train, test))
                                    
        return self.particiones
            
        
    
#####################################################################################################

class ValidacionBootstrap(EstrategiaParticionado):
    
    def __init__(self, n):
        super().__init__("Validacion Bootstrap con {} particiones".format(n))
        self.n = n

  # Crea particiones segun el metodo de boostrap
  # Devuelve una lista de particiones (clase Particion)
    def creaParticiones(self,datos,seed=None):    
        np.random.seed(seed)

        self.numeroParticiones = self.n

        # Generamos una permutacion de los indices
        indices = np.arange(datos.nDatos)
        self.particiones = []

        for i in range(self.n):

            # Generamos numeros aleatorios con repeticion
            aleatorios = np.random.randint(0, datos.nDatos, datos.nDatos)
            # Nos quedamos los ejemplos de los indices
            train = indices[aleatorios]
            # Obtenemos los indices que han sido excluidos
            excluidos = [i not in aleatorios for i in indices] 
            
            # El conjunto de indices esta formado por los indices excluidos
            test = indices[excluidos]

            self.particiones.append(Particion(train, test))

        return self.particiones


In [231]:
# Prueba de los metodos de particiones de la semana1

if __name__ == '__main__':
    
    
    dataset = Datos('../ConjuntosDatos/balloons.data')
    
    
    # Creamos una particion con validacion simple
    validacion1 = ValidacionSimple(0.8)
    particion1 = validacion1(dataset)
    
    # Creamos una particion con validacion cruzada
    k=4
    validacion2 = ValidacionCruzada(k)
    particion2 = validacion2(dataset)
    
    # Creamos una particion usando Validacion Bootstrap
    n=10
    validacion3 = ValidacionBootstrap(n)
    particion3 = validacion3(dataset)


In [305]:



class Clasificador:
  
    # Clase abstracta
    __metaclass__ = ABCMeta

    # Metodos abstractos que se implementan en casa clasificador concreto
    @abstractmethod
    # TODO: esta funcion deben ser implementadas en cada clasificador concreto
    # datosTrain: matriz numpy con los datos de entrenamiento
    # atributosDiscretos: array bool con la indicatriz de los atributos nominales
    # diccionario: array de diccionarios de la estructura Datos utilizados para la codificacion
    # de variables discretas
    def entrenamiento(self,datosTrain,atributosDiscretos,diccionario):
        pass


    @abstractmethod
    # TODO: esta funcion deben ser implementadas en cada clasificador concreto
    # devuelve un numpy array con las predicciones
    def clasifica(self,datosTest,atributosDiscretos,diccionario):
        pass


    # Obtiene el numero de aciertos y errores para calcular la tasa de fallo
    # TODO: implementar
    def error(self,datos,pred):
    # Aqui se compara la prediccion (pred) con las clases reales y se calcula el error    
        pass


    # Realiza una clasificacion utilizando una estrategia de particionado determinada
    # TODO: implementar esta funcion
    def validacion(self,particionado,dataset,clasificador,seed=None):

    # Creamos las particiones siguiendo la estrategia llamando a particionado.creaParticiones
    # - Para validacion cruzada: en el bucle hasta nv entrenamos el clasificador con la particion de train i
    # y obtenemos el error en la particion de test i
    # - Para validacion simple (hold-out): entrenamos el clasificador con la particion de train
    # y obtenemos el error en la particion test
        pass


    ##############################################################################

class ClasificadorNaiveBayes(Clasificador):


    def entrenamiento(self, datos, indices): 

        # Numero de atributos discretos
        self.nAtributosDiscretos = sum(datos.nominalAtributos) - 1
        self.nAtributos = len(datos.diccionarios) - 1
        self.nAtributosContinuos = self.nAtributos - self.nAtributosDiscretos       

        # Numero de clases (longitud del diccionario del campo clase)
        self.nClases = len(datos.diccionarios[-1])
        
        datosEntrenamiento = datos[indices]
        arrayClases = datosEntrenamiento[:,-1]
        nEntrenamiento = len(indices)
        
        # Maximo tam de posibles valores de atributos nominales
        self.vAtributos = max([len(d) for d in datos.diccionarios[:-1]])
        
        # Tablas de probabilidad a posteriori
        # Matrix con (indice Atributo Discreto, valor Atributo, Clase)
        self.posteriori = np.zeros((self.nAtributosDiscretos,self.vAtributos, self.nClases))
        self.gaussianas =  np.zeros((self.nAtributosContinuos,self.nClases, 2))
        # Mapeo entre indices reales y los indices de nuestras tablas
        self.indicesDiscretos = {}
        self.indicesContinuos = {}
        d_index = 0 # Cuenta de indices empleados discretos
        c_index = 0 # y continuos
        
        self.priori = np.empty(self.nClases)
        
        for c in range(self.nClases):
            self.priori[c] = len(np.where(arrayClases == c)[0])/ nEntrenamiento

        
               
        # Iteramos sobre los datos de entrenamiento (sin contar la clase)
        for k, discreto in enumerate(datos.nominalAtributos[:-1]):
            if discreto:
                self.indicesDiscretos[k] = d_index
                self._entrena_discreto(d_index, datosEntrenamiento[:,k], arrayClases, datos.diccionarios[k], nEntrenamiento)
                d_index += 1
            else:
                self.indicesContinuos[k] = c_index
                self._entrena_continuo(c_index, datosEntrenamiento[:,k], arrayClases)                
                c_index += 1

                
    def _entrena_discreto(self, k, datos, clase, diccionario, n):
        
        # Calculamos los conteos de atributos de cada clase
        repeticiones = np.empty(self.nClases, dtype=object)
        for c in range(self.nClases):
            repeticiones[c] = collections.Counter(datos[clase == c])

            
        # Correccion de laplace
        laplace = any(np.array([len(r) for r in repeticiones]) != len(diccionario))
        l= 1 if laplace else 0
        
        for c in range(self.nClases):
            for v in diccionario:
                atr = diccionario[v]
                self.posteriori[k][atr][c] = (l + repeticiones[c][atr]) / (n+l)
                
    def _entrena_continuo(self, k, datos, clase):
        
        for c in range(self.nClases):
            data = datos[clase == c]
            self.gaussianas[k,c,0] = np.mean(data)
            self.gaussianas[k,c,1] = np.std(data)
            
    def _probabilidadClase(self, dato, clase, datos):

        prob = self.priori[clase]
        print("dato",dato, "clase",clase)
        print("A priori", prob)
        
        for atr, nominal in enumerate(datos.nominalAtributos[:-1]): 
            if nominal:
                index = self.indicesDiscretos[atr]
                print("Indice atributo", index, "valor", dato[atr])
                prob *= self.posteriori[index,dato[atr],clase]
                print("posteriori", self.posteriori[index,dato[atr],clase])
            else:
                index = self.indicesContinuos[atr]
                prob *= norm.pdf(dato[atr], self.gaussianas[index,clase,0], 
                                self.gaussianas[index,clase,1])
            
        return prob

            

    def clasifica(self,datos, indices):
        
        
        clasificacion = np.full(len(indices), 99)
        probabilidades = np.zeros(self.nClases)
        
        
        for j, dato in enumerate(datos[indices]):
            #print("Dato", dato)
            for clase in range(self.nClases):
                #print("Clase", clase)
                probabilidades[clase] = self._probabilidadClase(dato, clase, datos)
                #print("La probs", probabilidades)
            clasificacion[j] = np.argmax(probabilidades)
            #print(clasificacion)
            
        return clasificacion

    
    ##############################################################################

class ScikitNaiveBayes(Clasificador):
    
    def entrenamiento(self, datos, indices): 
        
        # Calculamos las posteriori de los discretos
        self.NB = nb.MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
        # Almacenamos el conjunto de datos
        X = np.array(datos)
        # Almacenamos índices para seleccionar tanda
        Y = [i for i in indices]
        self.NB.fit(X,Y)

    def clasifica(self, datos, indices):
        
        print(self.NB.predict())     
                
    
      

In [306]:
dataset = Datos('../ConjuntosDatos/balloons.data')

c = ClasificadorNaiveBayes()
c.entrenamiento(dataset, range(len(dataset)))
clasificacion = c.clasifica(dataset, range(len(dataset)))

#c = ScikitNaiveBayes()
#c.entrenamiento(dataset, range(len(dataset)))

dato [1 1 1 0 1] clase 0
A priori 0.6
Indice atributo 0 valor 1
posteriori 0.3
Indice atributo 1 valor 1
posteriori 0.3
Indice atributo 2 valor 1
posteriori 0.238095238095
Indice atributo 3 valor 0
posteriori 0.238095238095
dato [1 1 1 0 1] clase 1
A priori 0.4
Indice atributo 0 valor 1
posteriori 0.2
Indice atributo 1 valor 1
posteriori 0.2
Indice atributo 2 valor 1
posteriori 0.428571428571
Indice atributo 3 valor 0
posteriori 0.428571428571
dato [1 1 1 0 1] clase 0
A priori 0.6
Indice atributo 0 valor 1
posteriori 0.3
Indice atributo 1 valor 1
posteriori 0.3
Indice atributo 2 valor 1
posteriori 0.238095238095
Indice atributo 3 valor 0
posteriori 0.238095238095
dato [1 1 1 0 1] clase 1
A priori 0.4
Indice atributo 0 valor 1
posteriori 0.2
Indice atributo 1 valor 1
posteriori 0.2
Indice atributo 2 valor 1
posteriori 0.428571428571
Indice atributo 3 valor 0
posteriori 0.428571428571
dato [1 1 1 1 0] clase 0
A priori 0.6
Indice atributo 0 valor 1
posteriori 0.3
Indice atributo 1 valor 1

In [304]:
#print(c.gaussianas)
print(c.posteriori)

[[[ 0.3         0.2       ]
  [ 0.3         0.2       ]]

 [[ 0.3         0.2       ]
  [ 0.3         0.2       ]]

 [[ 0.42857143  0.04761905]
  [ 0.23809524  0.42857143]]

 [[ 0.23809524  0.42857143]
  [ 0.42857143  0.04761905]]]
